<a href="https://colab.research.google.com/github/admantiumblack/hololive-datawarehouse-project/blob/main/vtuber_sna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install selenium==4.5.0
!pip install networkx==2.6.3
!pip install tqdm
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium.webdriver import common
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import matplotlib.pyplot as plt
from selenium import webdriver
from tqdm.auto import tqdm
import networkx as nx
import pandas as pd
import numpy as np
import random
import time
import os

result_directory = './result'
if not os.path.exists(result_directory):
    os.makedirs(result_directory)
BUFFER_CHOICE_LIST = np.arange(0.1, 3, 0.4).tolist()
USERNAME='kobokanaeru'
CSV_FILE='./kobo_tweets.csv'
random.seed(666)

In [ ]:
def buffer(multiplyer=1):
  time.sleep(random.choice(BUFFER_CHOICE_LIST) * multiplyer)

In [ ]:
USER_AGENT = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'
def define_webdriver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')  # runs browser in headless mode
  options.add_argument('--no-sandbox')
  options.add_argument("--disable-dev-shm-usage")
  options.add_argument('--ignore-certificate-errors')
  options.add_argument('--disable-gpu')
  options.add_argument('--log-level=3')
  options.add_argument('--disable-notifications')
  options.add_argument('--disable-popup-blocking')
  options.add_argument('--user-agent={}'.format(USER_AGENT))
  return webdriver.Chrome('chromedriver',chrome_options=options)
driver = define_webdriver()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [ ]:
df = pd.read_csv(CSV_FILE, encoding='utf-8', sep=';sep;')
df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,"""From-User""","""From-User-Id""","""To-User""","""To-User-Id""","""Language""","""Source""","""Text""","""Created-At""","""Geo-Location-Latitude""","""Geo-Location-Longitude""","""Retweet-Count""","""Id"""
0,"""Kobo Kanaeru (こぼ) ☔ @ホロライブID""","""1486629076005634049""",NaN,"""-1""","""in""","""<a href='http://twitter.com/download/iphone' ...","""Ga lagi” fine dining makan kerang yg disruput...",2022-10-01 22:47:59,NaN,NaN,165,"""1576237500082708480"""
1,"""Kobo Kanaeru (こぼ) ☔ @ホロライブID""","""1486629076005634049""",NaN,"""-1""","""in""","""<a href='http://twitter.com/download/iphone' ...","""Lagi nyoba baju” nya moona, etss dapet rok 😋""",2022-10-01 17:38:25,NaN,NaN,118,"""1576159593381535744"""
2,"""Kobo Kanaeru (こぼ) ☔ @ホロライブID""","""1486629076005634049""",NaN,"""-1""","""in""","""<a href='http://twitter.com/download/iphone' ...","""Pengen belajar fine dining biar ga norak http...",2022-10-01 17:02:00,NaN,NaN,308,"""1576150429360013312"""
3,"""Kobo Kanaeru (こぼ) ☔ @ホロライブID""","""1486629076005634049""",NaN,"""-1""","""in""","""<a href='http://twitter.com/download/iphone' ...","""SERU BANGET TADII WKWKWKWKWK https://t.co/lEW...",2022-10-01 13:04:26,NaN,NaN,137,"""1576090645843755009"""
4,"""Kobo Kanaeru (こぼ) ☔ @ホロライブID""","""1486629076005634049""",NaN,"""-1""","""tl""","""<a href='http://twitter.com/download/iphone' ...","""hehe https://t.co/05XdRijPdN""",2022-09-30 22:36:59,NaN,NaN,139,"""1575872342676283395"""
...,...,...,...,...,...,...,...,...,...,...,...,...
134,"""Kobo Kanaeru (こぼ) ☔ @ホロライブID""","""1486629076005634049""",NaN,"""-1""","""en""","""<a href='http://twitter.com/download/iphone' ...","""【WEEKLY SCHEDULE】 ☔️ Youtube channel: https:/...",2022-08-09 20:28:14,NaN,NaN,489,"""1556995772729946112"""
135,"""Kobo Kanaeru (こぼ) ☔ @ホロライブID""","""1486629076005634049""",NaN,"""-1""","""en""","""<a href='http://twitter.com/download/iphone' ...","""exercise brings happiness have a blessed day ...",2022-08-09 12:29:29,NaN,NaN,359,"""1556875292936785920"""
136,"""Kobo Kanaeru (こぼ) ☔ @ホロライブID""","""1486629076005634049""",NaN,"""-1""","""in""","""<a href='http://twitter.com/download/iphone' ...","""aku stream minecraft tapi banyak diemnya, gim...",2022-08-09 07:58:52,NaN,NaN,143,"""1556807188386963456"""
137,"""Kobo Kanaeru (こぼ) ☔ @ホロライブID""","""1486629076005634049""",NaN,"""-1""","""en""","""<a href='https://mobile.twitter.com' rel='nof...","""good night https://t.co/L2aWkaEYy9""",2022-08-08 22:21:29,NaN,NaN,671,"""1556661886178967560"""


In [ ]:
tweet_ids = df['"Id"'].str.replace('"', '')
tweet_ids

0      1576237500082708480
1      1576159593381535744
2      1576150429360013312
3      1576090645843755009
4      1575872342676283395
              ...         
134    1556995772729946112
135    1556875292936785920
136    1556807188386963456
137    1556661886178967560
138    1556654085805416451
Name: "Id", Length: 139, dtype: object

In [ ]:
def ensure_node_in_graph(graph, node):
  if node not in graph:
    graph.add_node(node)
  return graph

def add_edge(graph, start, end, weight=1):
  graph = ensure_node_in_graph(graph, start)
  graph = ensure_node_in_graph(graph, end)
  
  if end not in graph[start]:
    graph.add_edge(start, end, weight=0)
  graph[start][end]['weight'] += weight
  return graph

def get_next_sibling_element(driver, element):
  return driver.execute_script(
    'return arguments[0].nextElementSibling;',
    element
  )

def get_next_comment(driver, comment_element):
  next_comment_element = get_next_sibling_element(driver, comment_element)

  if next_comment_element.text == 'Show replies':
    while next_comment_element.text == 'Show replies':
      common.action_chains.ActionChains(driver).scroll_to_element(next_comment_element).scroll_by_amount(0, 100).perform()
      driver.get_screenshot_as_file('scrolled.png')
      next_comment_element.click()
      buffer(1.5)
      try:
        next_comment_element = get_next_sibling_element(driver, comment_element)
      except:
        print('element reference outdated, start from top')
        next_comment_element = driver.find_elements(common.by.By.XPATH, '//div[@data-testid="cellInnerDiv"]')[2]

  if next_comment_element.text == 'Show more replies' or get_next_sibling_element(driver, next_comment_element) is None:
    return None
  
  if next_comment_element.text == "You’re unable to view this Tweet because this account owner limits who can view their Tweets. Learn more":
    print('comment privated')
    next_comment_element = get_next_comment(driver, next_comment_element)
  
  if not next_comment_element.find_elements(common.by.By.TAG_NAME, 'article'):
    next_comment_element = get_next_sibling_element(driver, next_comment_element)
  
  return next_comment_element
  

def get_comment(driver, username, status_id):
  print(username, status_id)
  if not username.startswith('@'):
    username = '@' + username

  url = f'https://twitter.com/{username}/status/{status_id}'
  driver.get(url)
  element_present = EC.presence_of_element_located((By.XPATH, '//div[@data-testid="cellInnerDiv"]'))
  WebDriverWait(driver, 10).until(element_present)
  current_comment = driver.find_elements(common.by.By.XPATH, '//div[@data-testid="cellInnerDiv"]')[2]

  while current_comment is not None:
  # scroll down to comment to get infinity scrolling
    common.action_chains.ActionChains(driver).scroll_to_element(current_comment).scroll_by_amount(0, 10).perform()

    text_blob = current_comment.text.replace('\n', ' ')
    print(text_blob)
    # user_column = current_comment.find_element(common.by.By.CSS_SELECTOR, 'div[data-testid="User-Names"]')
    # user_name_column = user_column.find_element(common.by.By.CSS_SELECTOR, 'div[dir="ltr"]>span')
    user_name = current_comment.find_element(common.by.By.CSS_SELECTOR, 'a[role="link"]').get_attribute('href').split('/')
    user = '@' + user_name[-1]
    print(user)
    replying_to = [i.text for i in current_comment.find_elements(common.by.By.CSS_SELECTOR, 'a[dir="ltr"]')]
    yield username, replying_to

    try:
      current_comment = get_next_comment(driver, current_comment)
    except:
      current_comment = None
    buffer()


def create_status_sna_graph(generator_instance, graph):
  for node, edges in generator_instance():
    try:
      graph = ensure_node_in_graph(graph, node)
      for i in edges:
        graph = add_edge(graph, node, i)
    except:
      pass
  
  return graph


In [ ]:
def fetch_all_replies(username, status_ids):

  if not username.startswith('@'):
    username = '@' + username
  data_dict = {
      'username': [],
      'centrality': [],
      'density': [],
      'average_clustering_coef': [],
      'overall_reciprocity': [],
      'graphml_file': [],
      'id': []
  }
  for id in tqdm(status_ids):
    print(id)
    driver = define_webdriver()
    time.sleep(1)
    generator = lambda: get_comment(driver, username, id)
    graph = nx.DiGraph()
    try:
      graph = create_status_sna_graph(generator, graph)
    except Exception as e:
      print(e)
      driver.get_screenshot_as_file('error.png')
      driver.close()
      continue
    
    data_dict['username'].append(username)
    nx.write_graphml(graph, f'./result/{id}')
    data_dict['graphml_file'].append(id)
    data_dict['id'].append(id)
    
    try:
      data_dict['centrality'].append(nx.algorithms.eigenvector_centrality(graph, weight='weight')[username])
    except:
      data_dict['centrality'].append(None)
    
    try:
      data_dict['density'].append(nx.function.density(graph))
    except:
      data_dict['density'].append(None)
    
    try:
      data_dict['average_clustering_coef'].append(nx.algorithms.average_clustering(graph, weight='weight'))
    except:
      data_dict['average_clustering_coef'].append(None)
    
    try:
      data_dict['overall_reciprocity'].append(nx.algorithms.overall_reciprocity(graph))
    except:
      data_dict['overall_reciprocity'].append(None)
    
    driver.close()

  try:
    return pd.DataFrame(data_dict)
  except:
    return data_dict


In [ ]:
res_df = fetch_all_replies(USERNAME, tweet_ids[:67])

  0%|          | 0/67 [00:00<?, ?it/s]

1576237500082708480


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


@kobokanaeru 1576237500082708480
Kobo Kanaeru (こぼ) @ホロライブID @kobokanaeru · Oct 1 Replying to  @kobokanaeru Iya oyster yg pake diperes lemon itu 50 63 2,513
@kobokanaeru
Kobo Kanaeru (こぼ) @ホロライブID @kobokanaeru · Oct 1 Sumpah nanti aku cerita di stream dah, pengalaman banget lah pokoknya WKWKWKWK 124 165 3,574
@kobokanaeru
Kobo Kanaeru (こぼ) @ホロライブID @kobokanaeru · Oct 1 Udah lewat brp jam aku masih eneg dan skrng mulai mules, padahal cuman makan 1  Si Moona malah ga ketelen sama sekali Paling bener seblak lah buat perut ini 167 129 2,997
@kobokanaeru
Keebe The Kirby Plush @taranza_dark86 · Oct 1 Replying to  @kobokanaeru Yes
@taranza_dark86
Murasakiii (アヒヤ) @murasaki2502 · Oct 1 Replying to  @kobokanaeru Yg bener
@murasaki2502
hana @olattelv · Oct 1 Replying to  @kobokanaeru KENAPAA 3
@olattelv
SiPalingKeren @KerenKanGwej · Oct 1 Replying to  @kobokanaeru Oyster?
@KerenKanGwej
kevv @kevveru · Oct 1 Replying to  @kobokanaeru sakit perut bo? wkwk
@kevveru
Licteria Krikas @licteriakrikas · 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


...

@kobokanaeru 1566338887991042048
Teslaroot @teslaroot · Sep 4 Replying to  @kobokanaeru Murka donk 3
@teslaroot
Jesus Gaspar @JesusGaspar1305 · Sep 4 Replying to  @kobokanaeru Thank you for the stream Kobo, have an excellent day 4
@JesusGaspar1305
@Zero_02 @__Zero__two___ · Sep 4 Replying to  @kobokanaeru Sekarang serius? 2
@__Zero__two___
Zaid @Zaidxxxxxxx · Sep 4 Replying to  @kobokanaeru Hopefully it's nothing too bad for an unscheduled stream. 9
@Zaidxxxxxxx
Wafiq @WafiqHoshinova · Sep 4 Replying to  @kobokanaeru bikelah... 2
@WafiqHoshinova
Shiesuta @NayShiesuta · Sep 4 Replying to  @kobokanaeru Pasti mau ngomongin soal hubungan kita yah 2 30
@NayShiesuta
Axian @Axiaaan · Sep 4 Replying to  @NayShiesuta Bang.. Anda kan sudah sah sama Cornelius 8
@Axiaaan
Shena @shena6969 · Sep 4 Replying to  @kobokanaeru kobo aku panas dingin 3
@shena6969
Fadhil Gunatama @Fadhil13936P · Sep 4 Replying to  @kobokanaeru Ngeri ngeri 1
@Fadhil13936P
Rust @CRlNGEE · Sep 4 Replying to  @kobokanaeru Kaya

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


@kobokanaeru 1566152066011205638
Kobo Kanaeru (こぼ) @ホロライブID @kobokanaeru · Sep 3 Replying to  @kobokanaeru The stream 84 302 5,875
@kobokanaeru
Keebe The Kirby Plush @taranza_dark86 · Sep 3 Replying to  @kobokanaeru Live kobo reaction 1 1 121
@taranza_dark86
白竜 @Hakuryuu07 · Sep 3 Replying to  @taranza_dark86  and  @kobokanaeru 1 88
@Hakuryuu07
Das @Dasyu_sirkur · Sep 3 Replying to  @kobokanaeru Ni anak belum tidur 3
@Dasyu_sirkur
Zaid @Zaidxxxxxxx · Sep 3 Replying to  @kobokanaeru If I see a duo collab, I just assume its actually a 5 man. 135
@Zaidxxxxxxx
Liinette @adlynette_ · Sep 3 Replying to  @kobokanaeru yg waktu sm regis jg gitu kan wkwkwkwk 1 17
@adlynette_
Axian @Axiaaan · Sep 4 Replying to  @adlynette_ Iyaa wkwkwk tapi yg pas sama regis gk serame ini wkwkw soalnya cuman ada Rio sama astel.. Tapi tetep seru sih wkwkw.. Apalagi yg kobo 1v1 sama astel pake knife, tapi ujungnya malah di headshot  1 6
@Axiaaan
fidz @fidz_al_fidz · Sep 5 Replying to  @Axiaaan  and  @adlynette_ the 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


@kobokanaeru 1566021396467273728
meongskuy @meongskuyy · Sep 3 Replying to  @kobokanaeru Hari hari mengvalo 2 9
@meongskuyy
ємууу @ladymeliaa · Sep 3 Replying to  @meongskuyy  and  @kobokanaeru besok valo lagi ama regis,ollie,calli ama satu lagi lupa 1
@ladymeliaa
Fi @nonmitsua · Sep 3 Replying to  @ladymeliaa  @meongskuyy  and  @kobokanaeru Kobo, Ollie, Calli, Altare, Magni 1
@nonmitsua
meongskuy @meongskuyy · Sep 3 Replying to  @nonmitsua  @ladymeliaa  and  @kobokanaeru Sekarang lagi sama Fuma, Gamma, Rio, Axel 1
@meongskuyy
Fi @nonmitsua · Sep 3 Replying to  @meongskuyy  @ladymeliaa  and  @kobokanaeru Iya. Yg 5 itu senin pagi rasanya. 1 1
@nonmitsua
ємууу @ladymeliaa · Sep 3 Replying to  @nonmitsua  @meongskuyy  and  @kobokanaeru iya si regis yg spoiler haha
@ladymeliaa
notalle @kelincixubyy · Sep 3 Replying to  @kobokanaeru gas bgt gc 3
@kelincixubyy
Virus @Psychopompps · Sep 3 Replying to  @kobokanaeru Gas imo 1
@Psychopompps
abdee. @ridhoabdee_ · Sep 3 Replying to  @kobokanaeru l

In [ ]:
res_df.fillna(0).to_csv(f'{USERNAME}_result.csv')

In [ ]:
import shutil
shutil.make_archive(USERNAME, 'zip', result_directory)

'/content/kobokanaeru.zip'

In [ ]:
from google.colab import files
files.download(f'{USERNAME}.zip')
files.download(f'{USERNAME}_result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>